In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
from collections import OrderedDict

from torch.autograd import variable

import shutil

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
num_classes = 2 
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [4]:

data_dir = '/content/drive/MyDrive/Colab Notebooks/Malaria Detector Code/cell_images'
train_dir = data_dir + '/Train'
test_dir = data_dir + '/Test'


In [5]:
data_transforms= transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)


In [7]:

    
train_data = datasets.ImageFolder(train_dir, transform=data_transforms)
test_data = datasets.ImageFolder(test_dir, transform=data_transforms)


In [8]:
batch_size = 64
num_workers=0

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


In [9]:
#defining the pretrained network

neural_net=models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [10]:
#freeze feature paramters

for params in neural_net.parameters():
    params.requires_grad=True

In [11]:
classifier=nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 200)),
    ('relu', nn.ReLU()), 
    ('fc2', nn.Linear(200, 102)),
    ('drop', nn.Dropout(p=0.5)),
    ('output', nn.LogSoftmax(dim=1))
]))

In [12]:
neural_net.classfier=classifier


In [13]:
def model_save(state, is_best=False, filename='savepoint.pth'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'neural_net.pth')

In [ ]:
criterion= nn.NLLLoss()
optimiser= optim.Adam(neural_net.classfier.parameters(), lr=0.001)

cuda= torch.cuda.is_available()

if cuda:
    neural_net.cuda()
else:
    neural_net.cpu()
    
epochs=200
print_every=5
save_every=50
steps=0

for e in range(epochs):
    neural_net.train()
    running_loss=0
    accuracy_train=1
    
    for images, labels, in iter(train_loader):
        steps+=1
        inputs, labels=variable(images),variable(labels)
        
        optimiser.zero_grad()
        
        if cuda:
            inputs, labels=inputs.cuda(), labels.cuda()
            
        output=neural_net.forward(inputs)
        loss=criterion(output, labels)
        
        loss.backward()
        optimiser.step()
        
        running_loss+=loss.item()
        
        ps_train = torch.exp(output).data
        equality_train = (labels.data == ps_train.max(1)[1])
        accuracy_train += equality_train.type_as(torch.FloatTensor()).mean()
        
        
        
        if steps % print_every == 0:
            neural_net.eval()
            
            accuracy = 1
            valid_loss = 0
            
            for images, labels in test_loader:
                with torch.no_grad():
                    inputs = variable(images)
                    labels = variable(labels)

                    if cuda:
                        inputs, labels = inputs.cuda(), labels.cuda()

                    output = neural_net.forward(inputs)

                    valid_loss += criterion(output, labels).item()

                    ps = torch.exp(output).data
                    equality = (labels.data == ps.max(1)[1])

                    accuracy += equality.type_as(torch.FloatTensor()).mean()
                
            print("Epoch: {}/{}.. ".format(e+1, epochs), 
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}..".format(valid_loss/len(test_loader)),
                  "Training Accuracy: {:.3f}".format(accuracy_train/len(train_loader)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(test_loader)))
            
            running_loss = 0
            neural_net.train()
        if steps % save_every==0:
          print("Saving step number {}...".format(steps))
          state = {'state_dict': neural_net.classifier.state_dict(),
                     'optimizer' : optimiser.state_dict(),
                     'class_to_idx':train_data.class_to_idx}
          model_save(state)

            
     
            


/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:249: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")
/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:250: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(*args, **kwargs)


Epoch: 3/200..  Training Loss: 0.087..  Validation Loss: 0.321.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 5/200..  Training Loss: 0.152..  Validation Loss: 0.225.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 8/200..  Training Loss: 0.086..  Validation Loss: 0.399.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 10/200..  Training Loss: 0.156..  Validation Loss: 0.361.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 13/200..  Training Loss: 0.061..  Validation Loss: 0.338.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 15/200..  Training Loss: 0.175..  Validation Loss: 0.398.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 18/200..  Training Loss: 0.067..  Validation Loss: 0.343.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 20/200..  Training Loss: 0.145..  Validation Loss: 0.316.. Training Accuracy: 0.500 Validation Accuracy: 0.500
Epoch: 23/200..  Training Loss: 0.069..  Validation Loss: 0

In [ ]:


neural_net.eval()
criterion = nn.NLLLoss()


if cuda:
    neural_net.cuda()
else:
    neural_net.cpu()
    
accuracy = 0
test_loss = 0

for images, labels in test_loader:
    with torch.no_grad():
        inputs = variable(images)
        labels = variable(labels)

        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        output = neural_net.forward(inputs)

        test_loss += criterion(output, labels).item()

        ps = torch.exp(output).data
        equality = (labels.data == ps.max(1)[1])

        accuracy += equality.type_as(torch.FloatTensor()).mean()

print("Test Loss: {:.3f}..".format(test_loss/len(test_loader)),
      "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))


/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:249: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")
/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:250: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(*args, **kwargs)


Test Loss: 0.350.. Test Accuracy: 0.000


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Train', 'Test']:
            if phase == 'Train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for images, labels in train_data:
                inputs = images
                labels = labels

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                

            epoch_loss = running_loss / len(train_data)
            epoch_acc = running_corrects.double() / len(train_data)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'Test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
#Loading the pretrained RESNET Model

model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

torch.save(model_ft.state_dict(), 'malaria_detector.pt')


Epoch 0/14
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


RuntimeError: ignored